LIBRERIAS

In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import time

In [ ]:
# --- Carga de API Token (como antes) ---
load_dotenv()
api_token = os.getenv('TMDB_API_READ_ACCESS_TOKEN')
if not api_token:
    raise ValueError("Error: No se encontró TMDB_API_READ_ACCESS_TOKEN en el archivo .env")
print("API Token (v4) cargado exitosamente.")

# --- Lista de títulos de ejemplo para buscar ---
# (Más adelante, esto vendrá de tu archivo NetflixViewingHistory.csv)
example_titles = ["La casa de papel", "Forrest Gump", "Nuestro Planeta", "Título Inventado Que No Existe"]

# --- Lista para almacenar los resultados enriquecidos ---
enriched_data_list = []

# --- Headers para la API (constante) ---
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_token}"
}

# --- Iterar sobre los títulos de ejemplo ---
for title_to_search in example_titles:
    print(f"\n--- Procesando: {title_to_search} ---")
    
    # 1. BUSCAR con /search/multi
    search_url = "https://api.themoviedb.org/3/search/multi"
    search_params = {'query': title_to_search, 'language': 'es-ES'}
    best_match_id = None
    best_match_type = None
    
    try:
        response_search = requests.get(search_url, params=search_params, headers=headers)
        response_search.raise_for_status()
        search_results = response_search.json()

        # Encontrar el primer resultado relevante (movie/tv)
        for result in search_results.get('results', []):
            media_type = result.get('media_type')
            if media_type in ['movie', 'tv']:
                best_match_id = result.get('id')
                best_match_type = media_type
                print(f"Encontrado: ID={best_match_id}, Tipo={best_match_type}")
                break
        
        if not best_match_id:
            print(f"No se encontró coincidencia relevante para '{title_to_search}'.")
            continue # Pasar al siguiente título del bucle

    except requests.exceptions.RequestException as e:
        print(f"Error en búsqueda API para '{title_to_search}': {e}")
        continue # Pasar al siguiente título

    # Pequeña pausa entre búsqueda y detalles
    time.sleep(0.3) 

    # 2. OBTENER DETALLES si se encontró ID y Tipo
    if best_match_id and best_match_type:
        details_url = f"https://api.themoviedb.org/3/{best_match_type}/{best_match_id}"
        details_params = {'language': 'es-ES'}
        
        try:
            response_details = requests.get(details_url, params=details_params, headers=headers)
            response_details.raise_for_status()
            details = response_details.json()

            # 3. EXTRAER Y GUARDAR DATOS en un diccionario
            data_entry = {} # Diccionario para este título
            data_entry['search_query'] = title_to_search # Guardamos con qué buscamos
            data_entry['tmdb_id'] = best_match_id
            data_entry['media_type'] = best_match_type
            
            # Título y Fecha
            data_entry['fetched_title'] = details.get('title') if best_match_type == 'movie' else details.get('name', 'N/A')
            data_entry['release_date'] = details.get('release_date') if best_match_type == 'movie' else details.get('first_air_date', 'N/A')
            
            # Géneros
            genres_list = details.get('genres', [])
            data_entry['genres'] = [genre['name'] for genre in genres_list]
            
            # Duración (Runtime) - Lógica unificada
            runtime = 0 # Default
            if best_match_type == 'movie':
                runtime = details.get('runtime', 0)
            elif best_match_type == 'tv':
                episode_runtime = details.get('episode_run_time', [])
                if episode_runtime:
                    runtime = episode_runtime[0] # Tomamos el primer valor (más común)
            data_entry['runtime_minutes'] = runtime if runtime else 0 # Asegurar que sea numérico
            
            # Calificación
            data_entry['vote_average'] = details.get('vote_average', 0.0)
            
            # Poster
            data_entry['poster_path'] = details.get('poster_path', '')
            
            # Añadir el diccionario a nuestra lista
            enriched_data_list.append(data_entry)
            print(f"Datos para '{data_entry['fetched_title']}' añadidos.")

        except requests.exceptions.RequestException as e:
            print(f"Error obteniendo detalles API para ID {best_match_id}: {e}")
        except Exception as e:
            print(f"Error procesando detalles para ID {best_match_id}: {e}")

    # Pausa final antes de la siguiente iteración
    time.sleep(0.3)


# --- Convertir la lista de resultados a un DataFrame de Pandas ---
if enriched_data_list:
    results_df = pd.DataFrame(enriched_data_list)
    print("\n--- DataFrame con los Datos Enriquecidos (Ejemplo) ---")
    display(results_df) # 'display' funciona bien en Jupyter
else:
    print("\nNo se pudo enriquecer ningún dato en esta prueba.")

API Token (v4) cargado exitosamente.

--- Procesando: La casa de papel ---
Encontrado: ID=71446, Tipo=tv
Datos para 'La casa de papel' añadidos.

--- Procesando: Forrest Gump ---
Encontrado: ID=13, Tipo=movie
Datos para 'Forrest Gump' añadidos.

--- Procesando: Nuestro Planeta ---
Encontrado: ID=75782, Tipo=tv
Datos para 'Nuestro planeta (One Strange Rock)' añadidos.

--- Procesando: Título Inventado Que No Existe ---
